In [20]:
# Get Tiny Shakespeare from Karpythys repo
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


In [21]:

with open('input.txt', 'r', encoding='utf-8') as f:
    corpus = f.read()

In [22]:
print(len(corpus))

1115394


In [23]:
print(corpus[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [24]:
chars = sorted(list(set(corpus)))
vocab_size = len(chars)
print(''.join(chars))
print(vobac_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [25]:
char_to_idx = { char:idx for idx, char in enumerate(chars)}
idx_to_char = { idx:char for idx, char in enumerate(chars)}
encode = lambda chars: [char_to_idx[char] for char in chars]
decode = lambda idxs: ''.join([idx_to_char[idx] for idx in idxs])

In [26]:
# Add tiktotken encode later here to compare

In [27]:
print(encode("Hi, my name is Benjamin"))
print(decode(encode("Hi, my name is Benjamin")))

[20, 47, 6, 1, 51, 63, 1, 52, 39, 51, 43, 1, 47, 57, 1, 14, 43, 52, 48, 39, 51, 47, 52]
Hi, my name is Benjamin


In [28]:
import torch
data = torch.tensor(encode(corpus), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [29]:
split = int(0.9*len(data))
train_data = data[:split]
val_data = data[split:]

In [30]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [31]:
#Example of how the training will look

x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range (block_size):
    context = x[:i+1]
    predict = y[i]
    print(f"Following context {context} predicts the target : {predict}")

Following context tensor([18]) predicts the target : 47
Following context tensor([18, 47]) predicts the target : 56
Following context tensor([18, 47, 56]) predicts the target : 57
Following context tensor([18, 47, 56, 57]) predicts the target : 58
Following context tensor([18, 47, 56, 57, 58]) predicts the target : 1
Following context tensor([18, 47, 56, 57, 58,  1]) predicts the target : 15
Following context tensor([18, 47, 56, 57, 58,  1, 15]) predicts the target : 47
Following context tensor([18, 47, 56, 57, 58,  1, 15, 47]) predicts the target : 58


In [32]:
torch.manual_seed(42)

batch_size = 4
block_size = 8

def get_batch(split):
    #Build batch for train or eval
    data = train_data if split == 'train' else val_data
    idx = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return x, y


In [33]:
#Example batch

xb, yb = get_batch('train')
print("inputs")
print(xb.shape)
print(xb)
print("outputs")
print(yb.shape)
print(yb)

print('__________')

for b in range(batch_size): #num of batches
    for t in range(block_size): #step in time
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"Following context {context} predicts the target : {target}")


inputs
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
outputs
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
__________
Following context tensor([57]) predicts the target : 1
Following context tensor([57,  1]) predicts the target : 46
Following context tensor([57,  1, 46]) predicts the target : 47
Following context tensor([57,  1, 46, 47]) predicts the target : 57
Following context tensor([57,  1, 46, 47, 57]) predicts the target : 1
Following context tensor([57,  1, 46, 47, 57,  1]) predicts the target : 50
Following context tensor([57,  1, 46, 47, 57,  1, 50]) predicts the target : 53
Following context tensor([57,  1, 46, 47, 57,  1, 50, 53]) predicts the target : 60
Following context tensor([1]) predicts t

In [46]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)


        return logits, loss
    
    def generate(self, idx, max_new_tokens):
            #idx is (B, T) array of indices 
        for _ in range(max_new_tokens):
           
            logits, loss = self(idx)
            # focus only on last element
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # get sample form the prob - distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            # apppend new sample index to existing sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8865, grad_fn=<NllLossBackward0>)

uoiaF$z
M?kI;h
DbuMG,H3LYNmrDxKgTpvAKOF-jU.hc;fBMTGa-IS
g3lEb&ZQ,l;:m;lpcNN
KpVEYRIIM,'hCRbMAcWTkrnH


In [48]:

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [68]:
batch_size = 32

for steps in range(10000):

    #sample a batch
    xb, yb, = get_batch('train')

    #evaluate the loss

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.430663585662842


In [74]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))



The ine.
l t t tand ll meliseare totos ton! he my!

Youtor-findanjuthy t d withar m ous n INTEd limire art, ads!
OMESon thiepl cke me hon of sl qumak.
O and mor llad st cerfulor meceatio,
CLLes
E ye:


TEnd as hanus IIOn theavemarce

E:
I, prd gn,
ABcepoak,
LIXGURSO: n spen be tesis wod FLLIORD:

Qq
